In [1]:
#### Convert coffea output into root TH2's to feed into TUnfold
import coffea
import pickle
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import os
# import ROOT

In [12]:
%matplotlib inline
# fname = "coffeaOutput/dijetHists_QCDsim_pt200.0_eta2.4_NewHist.pkl"
fname = "coffeaOutput/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjec2016.pkl"
# fname = "coffeaOutput/trijetHists_QCDsim_pt200.0_eta2.4_bbloose.pkl"
# fname ="coffeaOutput/trijetHistsTest_JetHT_pt200.0_eta2.4_bbloose.pkl"
eras = ["2016", "2017", "2018", "2016APV"]
dir = "rootFiles/"
for year in eras:
    print(year)
    if year in fname:
        print(year)
        year_str = year
        break
    else:
        year_str = "ALL"
with open(fname, "rb") as f:
    result = pickle.load( f )
axis_names = [ax.name for ax in result['jet_pt_mass_reco_u'].axes]
print(axis_names)
cats = [cat for cat in result['jet_pt_mass_reco_u'][{'ptreco':sum, 'dataset':sum, 'mreco':sum}].axes[0]]
if "JetHT" in fname:
    if "dijet" in  fname:
        rootfname = 'dijetHistsJetHT_jec_' + year_str + '.root'
    elif "trijet" in fname:
        rootfname = 'trijetHistsJetHT_jec_' + year_str + '.root'
elif "QCD" in fname:
    if "dijet" in  fname:
        rootfname = 'dijetHistsQCDsim_jec_' + year_str + '.root'
    # integrate and sum over axes
    elif "trijet" in fname:
        rootfname = 'trijetHistsQCDsim_jec' + year_str + '.root'
if os.path.exists(dir + rootfname):
    rootfile = uproot.recreate(dir+rootfname)
else:
    rootfile = uproot.create(dir+rootfname)
    #### only add hists containing gen if SIM files
response_entries = {}
for syst in cats:
    rootfile['jet_ptreco_mreco_u_'+syst] = result['jet_pt_mass_reco_u'][{'dataset':sum, 'syst':syst}]
    rootfile['jet_ptreco_mreco_g_'+syst] = result['jet_pt_mass_reco_g'][{'dataset':sum, 'syst':syst}]
    if "QCD" in fname:
        rootfile['ptgen_mgen_u_'+syst] = result['response_matrix_u'][{'syst':syst}].project("ptgen", "mgen")
        rootfile['ptgen_mgen_g_'+syst] = result['response_matrix_g'][{'syst':syst}].project("ptgen","mgen")
        rootfile['ptreco_ptgen_u_'+syst] = result['response_matrix_u'][{'syst':syst}].project("ptreco", "ptgen")
        rootfile['ptreco_ptgen_g_'+syst] = result['response_matrix_g'][{'syst':syst}].project("ptreco","ptgen")
        rootfile['misses_ptgen_mgen_'+syst] = result['misses'][{'syst':syst}].project("ptgen", "mgen")
        rootfile['fakes_ptreco_mreco_'+syst] = result['fakes'][{'syst':syst}].project("ptreco","mreco")
        response_matrix_u_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy()
        response_matrix_g_values = result['response_matrix_g'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").values()
        print(response_matrix_g_values.shape)
        ptreco_centers = (ptreco_edges[:-1]+ptreco_edges[1:])/2
        print(ptreco_centers[np.newaxis])
        mreco_centers = (mreco_edges[:-1]+mreco_edges[1:])/2
        ptgen_centers = (ptgen_edges[:-1]+ptgen_edges[1:])/2
        mgen_centers = (mgen_edges[:-1]+mgen_edges[1:])/2
        print(np.shape(response_matrix_g_values))
        print(np.shape(response_matrix_g_values[np.newaxis]))
        response_entries.update({'groomed_'+syst:response_matrix_g_values[np.newaxis],
                       'ungroomed_'+syst:response_matrix_u_values[np.newaxis]})
        centers = {'ptreco':ptreco_centers[np.newaxis], 'mreco':mreco_centers[np.newaxis], 
                      'ptgen':ptgen_centers[np.newaxis], 'mgen':mgen_centers[np.newaxis]}
rootfile['response']=response_entries
rootfile['centers'] = centers
rootfile.close()

2016
2016
['dataset', 'syst', 'ptreco', 'mreco']
(10, 24, 10, 12)
[[ 240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.]]
(10, 24, 10, 12)
(1, 10, 24, 10, 12)
(10, 24, 10, 12)
[[ 240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.]]
(10, 24, 10, 12)
(1, 10, 24, 10, 12)
(10, 24, 10, 12)
[[ 240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.]]
(10, 24, 10, 12)
(1, 10, 24, 10, 12)
(10, 24, 10, 12)
[[ 240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.]]
(10, 24, 10, 12)
(1, 10, 24, 10, 12)
(10, 24, 10, 12)
[[ 240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.]]
(10, 24, 10, 12)
(1, 10, 24, 10, 12)


In [13]:
# fname = uproot.open('root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/280000/42B4DEA0-C0E7-A944-80DC-1D107A0F63FB.root')

In [14]:
# print(fname["Events"].keys("*HTXS*"))

In [17]:
hists = uproot.open('rootFiles/dijetHistsQCDsim_jec_2016.root')
print(hists['response'].keys())
resp_groomed = hists['response']['groomed_nominal'].array().to_numpy()[0]
print(hists['jet_ptreco_mreco_u_nominal'].to_numpy())
# print(np.shape(resp_groomed))
# print(np.max(resp_groomed))
# print(np.argwhere(resp_groomed==np.max(resp_groomed)))
# print(np.argwhere(resp_groomed.flatten()==np.max(resp_groomed)))
print(len(hists['centers']['mreco'].array().to_numpy()))

['groomed_nominal', 'ungroomed_nominal', 'groomed_jesUp', 'ungroomed_jesUp', 'groomed_jesDown', 'ungroomed_jesDown', 'groomed_jerUp', 'ungroomed_jerUp', 'groomed_jerDown', 'ungroomed_jerDown']
(array([[2.59318906e+04, 8.30690538e+03, 4.21332466e+04, 9.87471306e+04,
        4.00314609e+05, 1.39269134e+06, 1.43199515e+07, 5.93603547e+07,
        3.69119083e+08, 5.63961091e+08, 4.61671557e+08, 2.91708281e+08,
        1.75480524e+08, 1.07748679e+08, 6.50020774e+07, 3.56213416e+07,
        2.41579870e+07, 1.02974071e+06, 7.41180871e+03, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.11094466e+03, 1.85643655e+03, 9.97612472e+03, 1.83922031e+04,
        6.35571346e+04, 1.66389665e+05, 1.50669941e+06, 6.63227938e+06,
        5.46275738e+07, 1.18350974e+08, 1.33630530e+08, 1.06826991e+08,
        7.43005728e+07, 5.09084240e+07, 3.55620773e+07, 2.55918953e+07,
        3.60111467e+07, 1.05896690e+07, 1.17950119e+06, 4.02250323e+04,
        4.703